In [2]:
import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {
            "role": "user",
            "content": "한글로 답변해. 파이썬으로 'Hello, World!'를 출력하는 코드를 작성해줘."
        }
    ]
)

print(response.choices[0].message.content)

다음처럼 한 줄로 출력합니다.

print("Hello, World!")

실행 예: 파일로 저장한 뒤 python3 파일이름.py 로 실행하면 됩니다.


In [6]:
for choice in response.choices:
    print("=== Choice ===")
    print(choice.message.content)

=== Choice ===
다음처럼 한 줄로 출력합니다.

print("Hello, World!")

실행 예: 파일로 저장한 뒤 python3 파일이름.py 로 실행하면 됩니다.
